# Playground Interativo - Testando os Classificadores

**Execute este notebook APÓS treinar os modelos com `main.py`**

Aqui você vai:
- Testar os 3 modelos com seus próprios textos
- Ver predições e probabilidades lado a lado
- Comparar qual modelo é melhor para diferentes tipos de texto
- Experimentar e aprender interativamente!

## Setup

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from models.embedding_classifier import EmbeddingClassifier
from models.finetuned_classifier import FinetunedClassifier
from models.llm_classifier import LLMClassifier
from utils.data_loader import load_and_prepare_dataset, get_few_shot_examples
import config

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_style('whitegrid')

print("✓ Imports carregados!")

## 1. Carregar Modelos Treinados

Isso pode levar alguns segundos...

In [ ]:
print("Carregando modelos...\n")

# Carregar dataset para obter labels
train_df, test_df, id2label, label2id = load_and_prepare_dataset(max_samples=1000)
labels = list(id2label.values())

# 1. Embedding Classifier
print("[1/3] Carregando Embedding Classifier...")
embedding_clf = EmbeddingClassifier()
try:
    embedding_clf.load_model()
    print("      ✓ Modelo carregado de cache")
except:
    print("      ⚠️ Modelo não encontrado. Treinando do zero...")
    embedding_clf.fit(
        texts=train_df['text'].tolist(),
        labels=train_df['label'].tolist()
    )
    embedding_clf.save_model()
    print("      ✓ Modelo treinado e salvo")

# 2. Fine-tuned Classifier
print("\n[2/3] Carregando Fine-tuned Classifier...")
finetuned_clf = FinetunedClassifier(
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
)
try:
    finetuned_clf.load_model()
    print("      ✓ Modelo carregado de cache")
except:
    print("      ⚠️ Modelo não encontrado. Execute 'python main.py' primeiro.")
    print("      Ou aguarde enquanto treino (pode levar 30-60 min)...")
    finetuned_clf.train(
        train_texts=train_df['text'].tolist(),
        train_labels=train_df['label'].tolist(),
        epochs=1  # Apenas 1 época para teste rápido
    )
    finetuned_clf.save_model()
    print("      ✓ Modelo treinado e salvo")

# 3. LLM Classifier
print("\n[3/3] Carregando LLM Classifier...")
few_shot_examples = get_few_shot_examples(train_df, id2label, num_examples=3)
llm_clf = LLMClassifier(
    labels=labels,
    few_shot_examples=few_shot_examples
)
print("      ✓ LLM pronto (Gemini API)")

print("\n" + "="*60)
print("✅ Todos os modelos carregados e prontos!")
print("="*60)

## 2. Função para Testar um Texto

Esta função testa um texto em todos os 3 modelos

In [ ]:
def test_text(text: str, show_plot: bool = True):
    """
    Testa um texto em todos os 3 classificadores.
    
    Args:
        text: Texto para classificar
        show_plot: Se True, mostra gráfico de probabilidades
    """
    print("\n" + "="*80)
    print(f"Texto: {text}")
    print("="*80)
    
    results = {}
    
    # 1. Embedding
    pred_embedding = embedding_clf.predict([text])[0]
    proba_embedding = embedding_clf.predict_proba([text])[0]
    results['Embedding + KNN'] = {
        'prediction': id2label[pred_embedding],
        'probabilities': proba_embedding
    }
    
    # 2. Fine-tuned
    pred_finetuned = finetuned_clf.predict([text])[0]
    proba_finetuned = finetuned_clf.predict_proba([text])[0]
    results['Fine-tuned DistilBERT'] = {
        'prediction': id2label[pred_finetuned],
        'probabilities': proba_finetuned
    }
    
    # 3. LLM
    pred_llm = llm_clf.predict([text])[0]
    proba_llm = llm_clf.predict_proba([text])[0]
    results['LLM (Gemini)'] = {
        'prediction': labels[pred_llm],
        'probabilities': proba_llm
    }
    
    # Mostrar resultados
    print("\n📊 PREDIÇÕES:\n")
    
    for model_name, result in results.items():
        pred = result['prediction']
        proba = result['probabilities']
        confidence = proba.max() * 100
        
        print(f"{model_name:<25}: {pred.upper():<12} (confiança: {confidence:>5.1f}%)")
    
    # Verificar consenso
    predictions = [r['prediction'] for r in results.values()]
    
    if len(set(predictions)) == 1:
        print(f"\n✅ CONSENSO: Todos concordam que é '{predictions[0]}'")
    else:
        print("\n⚠️ DISCORDÂNCIA: Os modelos não concordam!")
    
    # Gráfico de probabilidades
    if show_plot:
        fig, axes = plt.subplots(1, 3, figsize=(16, 4))
        
        for ax, (model_name, result) in zip(axes, results.items()):
            proba = result['probabilities']
            pred = result['prediction']
            
            colors = ['green' if labels[i] == pred else 'lightblue' for i in range(len(labels))]
            
            ax.bar(labels, proba * 100, color=colors)
            ax.set_title(f"{model_name}\nPredição: {pred}")
            ax.set_ylabel('Probabilidade (%)')
            ax.set_ylim(0, 100)
            ax.tick_params(axis='x', rotation=45)
            
            # Adicionar valores no topo das barras
            for i, v in enumerate(proba * 100):
                ax.text(i, v + 2, f"{v:.0f}%", ha='center', va='bottom', fontsize=9)
        
        plt.tight_layout()
        plt.show()
    
    return results

print("✓ Função test_text() pronta!")

## 3. Textos de Exemplo

Vamos testar com alguns exemplos pré-definidos

In [ ]:
# Exemplos para cada classe
example_texts = {
    'joy': [
        "I am so happy today! Everything is going perfectly!",
        "This is the best day of my life, I feel amazing!",
    ],
    'sadness': [
        "I feel so lonely and empty inside. Nothing makes sense anymore.",
        "I'm heartbroken and don't know what to do.",
    ],
    'love': [
        "I cherish every moment we spend together. You mean everything to me.",
        "My heart fills with warmth whenever I think of you.",
    ],
    'anger': [
        "I'm so furious right now! This is completely unacceptable!",
        "I can't believe they did this to me. I'm absolutely livid!",
    ],
    'fear': [
        "I'm terrified of what might happen. My hands are shaking.",
        "The darkness scares me, I feel anxious and worried.",
    ],
    'surprise': [
        "Wow! I never expected this to happen! This is incredible!",
        "I'm shocked! This came completely out of nowhere!",
    ]
}

print("Exemplos de texto carregados!")
print(f"Total: {sum(len(v) for v in example_texts.values())} exemplos em {len(example_texts)} classes")

### Testar Exemplo: Joy (Alegria)

In [ ]:
test_text(example_texts['joy'][0])

### Testar Exemplo: Sadness (Tristeza)

In [ ]:
test_text(example_texts['sadness'][0])

### Testar Exemplo: Anger (Raiva)

In [ ]:
test_text(example_texts['anger'][0])

### Testar Exemplo: Fear (Medo)

In [ ]:
test_text(example_texts['fear'][0])

## 4. Teste com Seu Próprio Texto!

Agora é sua vez! Digite qualquer texto em inglês:

In [ ]:
# Digite seu texto aqui:
my_text = "I feel nervous about the presentation tomorrow"

test_text(my_text)

### ✏️ Experimente mais textos!

Modifique a célula acima e execute novamente com diferentes textos.

**Sugestões de testes:**
- Textos ambíguos (múltiplas emoções)
- Textos muito curtos vs muito longos
- Textos com sarcasmo ou ironia
- Textos em português (para ver como os modelos reagem)
- Textos neutros (sem emoção clara)

## 5. Teste em Lote

Testar múltiplos textos de uma vez

In [ ]:
def batch_test(texts: list[str]):
    """
    Testa múltiplos textos e mostra resumo.
    
    Args:
        texts: Lista de textos para testar
    """
    print(f"\nTestando {len(texts)} textos...\n")
    
    results_df = []
    
    for i, text in enumerate(texts, 1):
        print(f"[{i}/{len(texts)}] {text[:50]}...")
        
        # Predições
        pred_embedding = id2label[embedding_clf.predict([text])[0]]
        pred_finetuned = id2label[finetuned_clf.predict([text])[0]]
        pred_llm = labels[llm_clf.predict([text])[0]]
        
        # Verificar consenso
        predictions = [pred_embedding, pred_finetuned, pred_llm]
        consensus = len(set(predictions)) == 1
        
        results_df.append({
            'Texto': text[:60] + '...' if len(text) > 60 else text,
            'Embedding': pred_embedding,
            'Fine-tuned': pred_finetuned,
            'LLM': pred_llm,
            'Consenso': '✓' if consensus else '✗'
        })
    
    df = pd.DataFrame(results_df)
    
    print("\n" + "="*80)
    print("RESUMO DOS RESULTADOS")
    print("="*80)
    print(df.to_string(index=False))
    
    # Estatísticas
    consensus_count = df['Consenso'].value_counts().get('✓', 0)
    consensus_pct = consensus_count / len(df) * 100
    
    print("\n" + "="*80)
    print(f"Consenso: {consensus_count}/{len(df)} ({consensus_pct:.1f}%)")
    print("="*80)
    
    return df

print("✓ Função batch_test() pronta!")

### Testar Todos os Exemplos de uma Classe

In [ ]:
# Testar todos os exemplos de 'joy'
batch_test(example_texts['joy'])

### Testar TODOS os Exemplos

In [ ]:
# Flatten todos os exemplos
all_examples = [text for texts in example_texts.values() for text in texts]

batch_test(all_examples)

## 6. Casos Interessantes: Textos Ambíguos

Vamos testar textos que podem ter múltiplas interpretações

In [ ]:
ambiguous_texts = [
    "I can't believe this happened to me...",  # Surprise ou sadness?
    "This is just perfect.",  # Joy ou sarcasmo (anger)?
    "I'm overwhelmed by everything right now.",  # Fear, sadness ou surprise?
    "My heart is racing.",  # Fear, love ou surprise?
    "I don't know what to feel anymore.",  # Sadness ou confusion?
]

print("Testando textos ambíguos...\n")
print("Estes textos podem ter múltiplas interpretações.")
print("Vamos ver se os modelos concordam ou discordam.\n")

batch_test(ambiguous_texts)

### Análise Detalhada de um Caso Ambíguo

In [ ]:
# Escolher um texto ambíguo para análise detalhada
test_text(ambiguous_texts[0])

## 7. Comparação de Confiança

Qual modelo é mais "confiante" nas suas predições?

In [ ]:
def compare_confidence(texts: list[str]):
    """
    Compara a confiança (probabilidade máxima) de cada modelo.
    
    Args:
        texts: Lista de textos para testar
    """
    confidences = {'Embedding': [], 'Fine-tuned': [], 'LLM': []}
    
    for text in texts:
        # Predições com probabilidades
        proba_embedding = embedding_clf.predict_proba([text])[0]
        proba_finetuned = finetuned_clf.predict_proba([text])[0]
        proba_llm = llm_clf.predict_proba([text])[0]
        
        # Pegar confiança (max probability)
        confidences['Embedding'].append(proba_embedding.max())
        confidences['Fine-tuned'].append(proba_finetuned.max())
        confidences['LLM'].append(proba_llm.max())
    
    # Calcular médias
    avg_confidences = {k: np.mean(v) * 100 for k, v in confidences.items()}
    
    print("\n" + "="*60)
    print("CONFIANÇA MÉDIA POR MODELO")
    print("="*60)
    
    for model, conf in sorted(avg_confidences.items(), key=lambda x: x[1], reverse=True):
        print(f"{model:<15}: {conf:>5.1f}%")
    
    # Gráfico
    fig, ax = plt.subplots(figsize=(10, 6))
    
    positions = np.arange(len(texts))
    width = 0.25
    
    ax.bar(positions - width, np.array(confidences['Embedding']) * 100, width, label='Embedding', alpha=0.8)
    ax.bar(positions, np.array(confidences['Fine-tuned']) * 100, width, label='Fine-tuned', alpha=0.8)
    ax.bar(positions + width, np.array(confidences['LLM']) * 100, width, label='LLM', alpha=0.8)
    
    ax.set_xlabel('Texto')
    ax.set_ylabel('Confiança (%)')
    ax.set_title('Confiança dos Modelos por Texto')
    ax.set_xticks(positions)
    ax.set_xticklabels([f"Texto {i+1}" for i in range(len(texts))])
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim(0, 105)
    
    plt.tight_layout()
    plt.show()

# Testar com textos ambíguos
compare_confidence(ambiguous_texts)

### 🎯 Interpretação da Confiança

**Alta confiança (>80%):**
- Modelo está "certo" da resposta
- Texto tem sinais claros da emoção

**Média confiança (50-80%):**
- Modelo tem dúvidas
- Texto pode ser ambíguo

**Baixa confiança (<50%):**
- Modelo está "chutando"
- Texto muito ambíguo ou fora do domínio de treino

**Modelo sempre muito confiante:**
- Pode indicar overfitting
- Ou modelo muito simplista

**Modelo sempre pouco confiante:**
- Pode precisar de mais treinamento
- Ou dados de teste muito diferentes do treino

## 8. Análise Exploratória Livre

Use esta célula para seus próprios experimentos!

In [ ]:
# Espaço para seus experimentos

# Exemplo: Testar textos em português
portuguese_texts = [
    "Estou muito feliz hoje!",
    "Estou triste e sozinho.",
    "Eu te amo muito!",
]

print("\n⚠️ AVISO: Modelos foram treinados em INGLÊS!")
print("Resultados em português serão ruins (esperado).\n")

batch_test(portuguese_texts)

## 9. Resumo e Conclusões

### 🎓 O que aprendemos?

1. **Consenso ≠ Correto**: Todos concordarem não garante que estão certos
2. **Confiança ≠ Precisão**: Um modelo confiante pode estar errado
3. **Contexto importa**: Mesma palavra pode ter emoções diferentes
4. **Ambiguidade é real**: Nem sempre há uma resposta "certa"

### 📊 Próximos Passos

1. **Teste mais textos** do seu domínio de interesse
2. **Identifique padrões**: Quando cada modelo é melhor?
3. **Ajuste o modelo**: Use insights para melhorar treinamento
4. **Colete dados**: Salve casos interessantes para análise

### 🚀 Dicas para Produção

- **Use ensemble**: Combine predições dos 3 modelos
- **Defina threshold**: Rejeite predições com baixa confiança
- **Monitore**: Salve casos onde modelos discordam
- **Feedback loop**: Use correções de usuário para re-treinar

## 10. Exportar Seus Testes

Salvar seus experimentos para referência futura

In [ ]:
# Exemplo: Salvar resultados de batch test

# my_texts = [...] # Seus textos
# results_df = batch_test(my_texts)

# Salvar em CSV
# output_path = Path('../results/my_experiments.csv')
# results_df.to_csv(output_path, index=False)
# print(f"✓ Resultados salvos em: {output_path}")

print("Use as células acima para salvar seus experimentos!")